#**Chat with Multiple PDF's Llama 2 + Pinecone + LangChain**

#**Step: 01: Install All the Required Packages**

In [1]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=deea0a85ed111acf6f3ad1646093abe066e1e6957f135bab411509c3b14efe63
  Stored in directory: /root/.cache/pip/wheels/62/f2

In [ ]:
#pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te

#**Step 2: Import All the Required Libraries**

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

#**Step 3: Load the PDF Files**

In [15]:
#loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Qasper_Archive_First_set_100_Papers")
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Set1_100_Papers/EVAL_question_paper")
data = loader.load()

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
print(data)

[Document(page_content='Learning to Compose Neural Networks for Question Answering\nJacob Andreas andMarcus Rohrbach andTrevor Darrell andDan Klein\nDepartment of Electrical Engineering and Computer Sciences\nUniversity of California, Berkeley\n{jda,rohrbach,trevor,klein }@eecs.berkeley.edu\nAbstract\nWe describe a question answering model that\napplies to both images and structured knowl-\nedge bases. The model uses natural lan-\nguage strings to automatically assemble neu-\nral networks from a collection of composable\nmodules. Parameters for these modules are\nlearned jointly with network-assembly param-\neters via reinforcement learning, with only\n(world, question, answer) triples as supervi-\nsion. Our approach, which we term a dynamic\nneural module network , achieves state-of-the-\nart results on benchmark datasets in both vi-\nsual and structured domains.\n1 Introduction\nThis paper presents a compositional, attentional\nmodel for answering questions about a variety of\nworld 

#**Step 04: Extract the text from the PDF's**

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

#**Step 05: Split the Extracted Data into Text Chunks**

In [19]:
docs = text_splitter.split_documents(data)

In [20]:
print(len(docs))

3214


In [21]:
docs[34]

Document(page_content='(a) A neural find module for computing an attention over\npixels. (b) The same operation applied to a knowledge base.\n(c) Using an attention produced by a lower module to identify', metadata={'source': '/content/drive/MyDrive/Set1_100_Papers/EVAL_question_paper/Copy of 1601.01705.pdf', 'page': 1})

#**Step 06: Downlaod the Embeddings From the Hugging Face**

In [22]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [23]:
query_result = embeddings.embed_query("This is a test query")

In [24]:
print("Length", len(query_result))

Length 384


#**Step 07: Initializing the Pinecone**

In [25]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'c5742382-d6c6-435c-a821-d5d834f172a9')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [26]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "ncu" # put in the name of your pinecone index her

#**Step 08: Create Embeddings for each of the Text Chunk**

In [ ]:
#docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

#If you already have an index, you can load it like this

In [27]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

#**Step 09: Similarity Search**

In [28]:
query = "In the paper The Grail theorem prover: Type theory for syntax and semantics, What formalism does Grail use?"

In [29]:
docs = docsearch.similarity_search(query, k=4)

In [30]:
docs

[Document(page_content='9 The Grail theorem prover: Type theory for syntax and semantics 9\nGiven a semantic analysis without a corresponding syntactic proof, we can try'),
 Document(page_content='9 The Grail theorem prover: Type theory for syntax and semantics 9\nGiven a semantic analysis without a corresponding syntactic proof, we can try'),
 Document(page_content='9 The Grail theorem prover: Type theory for syntax and semantics 9\nGiven a semantic analysis without a corresponding syntactic proof, we can try'),
 Document(page_content='Chapter 9\nThe Grail theorem prover:\nType theory for syntax and semantics\nRichard Moot\nAbstract Type-logical grammars use a foundation of logic and type theory to model')]

#**Step 10: Creating a Llama2 Model Wrapper**

In [31]:
from huggingface_hub import notebook_login
import torch

In [32]:
notebook_login()

In [33]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
#!pip install accelerate
#!pip install bitsandbytes

In [34]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float32,
                                             use_auth_token=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [39]:
# Create a text generation pipeline
from transformers import pipeline, set_seed
pipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

#**Step 11: Create a Prompt Template**

In [41]:
#SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
#If you don't know the answer, just say that you don't know, don't try to make up an answer."""
#SYSTEM_PROMPT = """You will be provided with the names of research papers, and your task is to generate coherent and detailed paragraphs summarizing key aspects of each paper. For questions about the central argument, focus on extracting the primary point and overarching theme of the paper. When summarizing, concisely capture the main findings, conclusions, and relevant details. If you're unfamiliar with the paper, respond with 'I don't know' and consider adding a contextually appropriate remark. Aim for a fluid and informative paragraph style, avoiding a strict 'question and answer' format."""
SYSTEM_PROMPT = """
Given the diverse range of academic papers, your task is to provide insightful and accurate responses to questions related to various research articles. Your answers should demonstrate a comprehensive understanding of the paper's content and key findings. Questions will start with the paper's name, where your initial response should include the author names. Subsequent questions will delve into specific details of that particular paper.If you're unfamiliar with the paper, respond with 'I don't know' and consider adding a contextually appropriate remark.

**Instructions for Model:**
1. Read the prompts carefully and ensure your responses directly address the questions asked.
2. Pay attention to details such as experimental methodologies, key arguments, and specific examples mentioned in the papers.
3. Provide concise and informative answers, avoiding vague or generic responses.
4. If a question refers to a specific paper, generate your response based on the content of that paper, making sure to incorporate relevant details.
"""


In [42]:
#B_INST, E_INST = "[INST]", "[/INST]"
#B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

#SYSTEM_PROMPT = [B_SYS + prompt + E_SYS for prompt in SYSTEM_PROMPT]
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [43]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [44]:
#instruction = """
#{context}

#Question: {question}
#"""

instruction = """
{context}

Question: {question}
"""

In [45]:
#template = B_INST + '\n'.join(SYSTEM_PROMPT) + instruction + E_INST
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [46]:
template

"[INST]<<SYS>>\n\nGiven the diverse range of academic papers, your task is to provide insightful and accurate responses to questions related to various research articles. Your answers should demonstrate a comprehensive understanding of the paper's content and key findings. Questions will start with the paper's name, where your initial response should include the author names. Subsequent questions will delve into specific details of that particular paper.If you're unfamiliar with the paper, respond with 'I don't know' and consider adding a contextually appropriate remark.\n\n**Instructions for Model:**\n1. Read the prompts carefully and ensure your responses directly address the questions asked.\n2. Pay attention to details such as experimental methodologies, key arguments, and specific examples mentioned in the papers.\n3. Provide concise and informative answers, avoiding vague or generic responses.\n4. If a question refers to a specific paper, generate your response based on the conte

In [47]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [48]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [49]:
result = qa_chain("In the paper The Grail theorem prover: Type theory for syntax and semantics, What formalism does Grail use?")

#**Results for a single query**

In [50]:
result['result']

'  In the paper "The Grail theorem prover: Type theory for syntax and semantics," the authors use a formalism called "type theory" to provide a foundation for the design and implementation of a theorem prover. Specifically, Grail is based on the Calculus of Inductive Constructions (CIC), which is a type theory that allows for the expression of complex mathematical structures through a series of inductive definitions. (Source: Page 2 of the paper)'

#**Results for Multiple queries**

In [51]:
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result']}")

prompt:In the paper Paraphrase Generation from Latent-Variable PCFGs for Semantic Parsing, How many paraphrases are generated per question?
Answer:  According to the paper "Paraphrase Generation from Latent-Variable PCFGs for Semantic Parsing" by Shashi Narayan, Siva Reddy, and Shay B. Cohen, the authors generate a diverse set of paraphrases for each question in the semantic parsing task. They do not provide a specific number of paraphrases generated per question, as the number of paraphrases can vary depending on the complexity of the question and the model's performance. However, they report that their model is able to generate a large number of paraphrases for each question, with an average of 10-15 paraphrases per question in their experimental evaluation.
prompt:In the paper Paraphrase Generation from Latent-Variable PCFGs for Semantic Parsing, What latent variables are modeled in the PCFG?
Answer:  According to the paper "Paraphrase Generation from Latent-Variable PCFGs for Seman

KeyboardInterrupt: ignored